In [1]:
import os
import pandas as pd
import re
import csv

chunks_folder = "chunks_2"  # Your chunks folder
chunks_by_pmid = {}

# Step 1: Load all chunks and group them by PMID
for csv_file in os.listdir(chunks_folder):
    if csv_file.endswith(".csv"):
        pmid = csv_file.split('_')[0]
        csv_path = os.path.join(chunks_folder, csv_file)
        df = pd.read_csv(csv_path)
        df['PMID'] = pmid
        if pmid not in chunks_by_pmid:
            chunks_by_pmid[pmid] = []
        chunks_by_pmid[pmid].append(df)

# Step 2: Concatenate all DataFrames per PMID into one DataFrame
chunks_by_pmid = {pmid: pd.concat(dfs, ignore_index=True) for pmid, dfs in chunks_by_pmid.items()}

# Step 3: Create batches of 2 PDFs
pmid_list = list(chunks_by_pmid.keys())
batch_size = 2

batches = [pmid_list[i:i + batch_size] for i in range(0, len(pmid_list), batch_size)]

top_n = 10  # or however many top chunks you want to retrieve per batch

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch

# Initialize accelerator and get device
accelerator = Accelerator()
device = accelerator.device
model_name = "mistralai/Mistral-7B-v0.1"
token = "your_token"

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_enable_fp32_cpu_offload=True
)

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token=token,
    cache_dir="/data/gent/490/vsc49096/huggingface"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=token,
    cache_dir="/data/gent/490/vsc49096/huggingface",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)

# Ensure pad token is set correctly
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # or any other token you want to use as pad_token

model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
#pip uninstall peft -y
#pip install git+https://github.com/huggingface/peft.git
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer

# Load the model on CPU to avoid CUDA memory errors
model_st = SentenceTransformer('intfloat/multilingual-e5-large-instruct', device='cuda')

# Function to encode text on CPU
def embed_text(text, model):
    embeddings = model.encode(text, convert_to_tensor=True, device='cuda')
    return embeddings

query_texts = [
    """You are a biomedical research assistant specializing in extracting protein-protein interactions from scientific articles. nly extract information that is directly stated in the text. Do not invent interactions and only list what the text supports. From the text below, identify and extract all explicitly stated protein-protein interactions.
     Output a list of interactions in the following structured format: 
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     "
     """
     ,
     
     """You are tasked with extracting protein-protein interactions from biomedical literature.
     For each interaction you extract, assign a confidence score based on how explicitly the interaction is described (High, Medium, Low). Focus on accuracy and clear evidence from the text.
     Structure your output as follows:
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Confidence Level: [high/medium/low]
     Evidence: [direct quotation from text]
     "
     """
     ,
     """You are a biomedical research assistant specializing in extracting protein-protein interactions from scientific articles. From the text below, identify and extract all explicitly stated protein-protein interactions.
     Output a list of interactions in the following structured format: 
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     "
     
     Example 1:
     Text: "The interaction between p53 and MDM2 is crucial for regulating the cell cycle. p53 binds to MDM2 to inhibit its activity."
     Extraction:
     "
     Protein p53 -> inhibits -> Protein MDM2
     Evidence: "p53 binds to MDM2 to inhibit its activity.
     "
    
     Example 2
     Text: "EGFR forms a complex with GRB2 following activation by EGF."
     Extraction:
     "
     Protein EGFR -> complex formation -> Protein GRB2
     Evidence: "EGFR forms a complex with GRB2 following activation by EGF.
     "
     """
     ,
     """You are tasked with extracting protein-protein interactions from biomedical literature.
     For each interaction you extract, assign a confidence score based on how explicitly the interaction is described (High, Medium, Low). Focus on accuracy and clear evidence from the text.
     Structure your output as follows:
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     Confidence Level: [high/medium/low]
     "

     Example 1:
     Text: "Upon DNA damage, ATM phosphorylates and activates CHEK2, promoting checkpoint arrest."
     Extraction:
     "
     Protein ATM -> activates via phosphorylation -> Protein CHEK2
     Evidence: "ATM phosphorylates and activates CHEK2"
     Confidence: High
     "

     Example 2:
     Text: "Studies suggest that MYC may cooperate with MAX to drive transcriptional activation, although their interaction was not directly tested."
     Extraction:
     "
     Protein MYC -> cooperates -> Protein MAX
     Evidence: "MYC may cooperate with MAX"
     Confidence: Medium
     "
     """
]



for prompt_index, query_text in enumerate(query_texts, start=1):
    print(f"\n🧪 Testing Prompt {prompt_index}: {query_text}")

    query_embedding = embed_text(query_text, model_st)
    query_embedding_np = query_embedding.cpu().numpy().reshape(1, -1)

    ppi_results = []  # clear previous results

    for batch_num, pmid_batch in enumerate(batches, start=1):
        print(f"\n🔄 Processing batch {batch_num} with PMIDs: {pmid_batch}")

        batch_df = pd.concat([chunks_by_pmid[pmid] for pmid in pmid_batch], ignore_index=True)

        batch_embeddings = np.vstack(
            batch_df['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep=',')).values
        )

        cosine_similarities = cosine_similarity(query_embedding_np, batch_embeddings)

        top_indices = cosine_similarities[0].argsort()[-top_n:][::-1]
        top_chunks = batch_df.iloc[top_indices].copy()
        top_chunks['cosine_similarity'] = cosine_similarities[0][top_indices]

        print(f"Top {top_n} relevant chunks from batch {batch_num}:")
        print(top_chunks[['page_number', 'sentence_chunk', 'cosine_similarity', 'PMID']])

        for i, row in top_chunks.iterrows():
            context = row["sentence_chunk"]
            pmid = row["PMID"]

            prompt = f"Question: {query_text}\nContext: {context}\nAnswer:"

            inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

            with torch.no_grad():
                output = model.generate(
                    **inputs,
                    max_new_tokens=200,
                    do_sample=False,
                    top_k=50,
                    temperature=0.7
                )

            answer = tokenizer.decode(output[0], skip_special_tokens=True)

            print(f"\n📄 PMID: {pmid}, Page: {row['page_number']}")
            print(f"📝 Prompt: {prompt}")
            print(f"🧠 Answer: {answer}")

            matches = re.findall(r'([\w\-]+)\s*->\s*\[?([\w\s\-]+)\]?\s*->\s*([\w\-]+)', answer)
            for match in matches:
                protein_1, interaction_type, protein_2 = match
                ppi_results.append({
                    "PMID": pmid,
                    "protein_1": protein_1,
                    "interaction_type": interaction_type.strip(),
                    "protein_2": protein_2
                })

    # Save CSV after all batches are processed for this prompt
    output_file = f"ppi_predictions_prompt_{prompt_index}.csv"
    os.makedirs("ppi_outputs", exist_ok=True)
    output_path = os.path.join("ppi_outputs", output_file)

    with open(output_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["PMID", "protein_1", "interaction_type", "protein_2"])
        writer.writeheader()
        writer.writerows(ppi_results)

    print(f"✅ Saved results for Prompt {prompt_index} to {output_path}")


🧪 Testing Prompt 1: You are a biomedical research assistant specializing in extracting protein-protein interactions from scientific articles. nly extract information that is directly stated in the text. Do not invent interactions and only list what the text supports. From the text below, identify and extract all explicitly stated protein-protein interactions.
     Output a list of interactions in the following structured format: 
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     "
     


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



🔄 Processing batch 1 with PMIDs: ['7609053']
Top 10 relevant chunks from batch 1:
    page_number                                     sentence_chunk  \
14            4  4A, lanes 1 and 2). This complex was signiﬁcan...   
38            7  Octamer transcription factors and the cell typ...   
52            8  Xiao, J. H., I. Davidson, D. Ferrandon, R. Ros...   
49            8  T-cell-induced expression of human immunodeﬁci...   
31            7  Folks, T. M., J. Justement, A. Kinter, C. A. D...   
50            8  12:1043– 1053. 68. Tong-Starksen, S. E., T. M....   
18            5  A search of the Transcription Factor Database ...   
44            8  Science 265: 1587–1590. 53. Moses, A. V., C. I...   
47            8  Poli, G., P. Bressler, A. Kinter, E. Duh, W. C...   
41            8  46. Lu, Y., N. Touzjian, M. Stenzel, T. Dorfma...   

    cosine_similarity     PMID  
14           0.826904  7609053  
38           0.824838  7609053  
52           0.824297  7609053  
49           0

/user/gent/490/vsc49096/.local/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📄 PMID: 7609053, Page: 4
📝 Prompt: Question: You are a biomedical research assistant specializing in extracting protein-protein interactions from scientific articles. nly extract information that is directly stated in the text. Do not invent interactions and only list what the text supports. From the text below, identify and extract all explicitly stated protein-protein interactions.
     Output a list of interactions in the following structured format: 
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     "
     
Context: 4A, lanes 1 and 2). This complex was signiﬁcantly diminished by competition with an excess of unlabeled peri-kB oligonu- cleotide (lane 3) and to a lesser degree by the slightly shorter peri-kB(C) oligonucleotide (lane 4) but not by the kB site, a mutant kB site, or the PuB1 site of the HIV-2 enhancer (lanes 5 to 7). With HL-60 nuclear extracts, there was again a speciﬁc complex seen which w

...


📄 PMID: 7609053, Page: 8
📝 Prompt: Question: You are tasked with extracting protein-protein interactions from biomedical literature.
     For each interaction you extract, assign a confidence score based on how explicitly the interaction is described (High, Medium, Low). Focus on accuracy and clear evidence from the text.
     Structure your output as follows:
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     Confidence Level: [high/medium/low]
     "

     Example 1:
     Text: "Upon DNA damage, ATM phosphorylates and activates CHEK2, promoting checkpoint arrest."
     Extraction:
     "
     Protein ATM -> activates via phosphorylation -> Protein CHEK2
     Evidence: "ATM phosphorylates and activates CHEK2"
     Confidence: High
     "

     Example 2:
     Text: "Studies suggest that MYC may cooperate with MAX to drive transcriptional activation, although their interaction was not directly tested."
     

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📄 PMID: 7609053, Page: 7
📝 Prompt: Question: You are tasked with extracting protein-protein interactions from biomedical literature.
     For each interaction you extract, assign a confidence score based on how explicitly the interaction is described (High, Medium, Low). Focus on accuracy and clear evidence from the text.
     Structure your output as follows:
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     Confidence Level: [high/medium/low]
     "

     Example 1:
     Text: "Upon DNA damage, ATM phosphorylates and activates CHEK2, promoting checkpoint arrest."
     Extraction:
     "
     Protein ATM -> activates via phosphorylation -> Protein CHEK2
     Evidence: "ATM phosphorylates and activates CHEK2"
     Confidence: High
     "

     Example 2:
     Text: "Studies suggest that MYC may cooperate with MAX to drive transcriptional activation, although their interaction was not directly tested."
     

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



📄 PMID: 7609053, Page: 7
📝 Prompt: Question: You are tasked with extracting protein-protein interactions from biomedical literature.
     For each interaction you extract, assign a confidence score based on how explicitly the interaction is described (High, Medium, Low). Focus on accuracy and clear evidence from the text.
     Structure your output as follows:
     "
     Protein [name] -> [Interaction type if available] -> Protein [name]
     Evidence: [direct quotation from text]
     Confidence Level: [high/medium/low]
     "

     Example 1:
     Text: "Upon DNA damage, ATM phosphorylates and activates CHEK2, promoting checkpoint arrest."
     Extraction:
     "
     Protein ATM -> activates via phosphorylation -> Protein CHEK2
     Evidence: "ATM phosphorylates and activates CHEK2"
     Confidence: High
     "

     Example 2:
     Text: "Studies suggest that MYC may cooperate with MAX to drive transcriptional activation, although their interaction was not directly tested."
     